Теперь попробую подходы, основанные на информации о фильмах.

In [4]:
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

genres = pd.read_csv('movie.csv')

In [75]:
genres[genres.genres == 'Adventure']

,movieId,title,genres
924,941,"Mark of Zorro, The (1940)",Adventure
1050,1070,Macao (1952),Adventure
1766,1849,Prince Valiant (1997),Adventure
2392,2477,Firewalker (1986),Adventure
2452,2537,Beyond the Poseidon Adventure (1979),Adventure
...,...,...,...
26071,125173,Tom Sawyer (2011),Adventure
26109,125543,Against The Sun (2015),Adventure
26658,128247,The Corsican Brothers (1941),Adventure
27050,129978,Mission Bloody Mary (1965),Adventure


In [6]:
scores = pd.read_csv('genome_scores.csv')
scores.head()

,movieId,tagId,relevance
0,1,1,0.02500
1,1,2,0.02500
2,1,3,0.05775
3,1,4,0.09675
4,1,5,0.14675


In [10]:
tags = pd.read_csv('genome_tags.csv')
tags.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1128 entries, 0 to 1127
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   tagId   1128 non-null   int64 
 1   tag     1128 non-null   object
dtypes: int64(1), object(1)
memory usage: 17.8+ KB


In [9]:
links = pd.read_csv('link.csv')
links.head()

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0


In [13]:
tags1 = pd.read_csv('tag.csv')
tags1.head()

,userId,movieId,tag,timestamp
0,18,4141,Mark Waters,2009-04-24 18:19:40
1,65,208,dark hero,2013-05-10 01:41:18
2,65,353,dark hero,2013-05-10 01:41:19
3,65,521,noir thriller,2013-05-10 01:39:43
4,65,592,dark hero,2013-05-10 01:41:18


In [37]:
films_tags = tags1.drop(columns=['userId', 'timestamp']).groupby('movieId', as_index=False).agg(lambda y: ' '.join(y))

In [39]:
films_tags.head()

,movieId,tag
0,1,Watched computer animation Disney animated fea...
1,2,time travel adapted from:book board game child...
2,3,old people that is actually funny sequel fever...
3,4,chick flick revenge characters chick flick cha...
4,5,Diane Keaton family sequel Steve Martin weddin...


In [40]:
tfidf = TfidfVectorizer(stop_words='english')
films_tags.tag.fillna('', inplace=True)
overview_matrix = tfidf.fit_transform(films_tags.tag)
overview_matrix.shape, films_tags.movieId.shape

((19545, 23863), (19545,))

In [41]:
similarity_matrix = linear_kernel(overview_matrix, overview_matrix)
similarity_matrix

array([[1.00000000e+00, 7.14348534e-02, 1.10349442e-02, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [7.14348534e-02, 1.00000000e+00, 9.41112516e-04, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [1.10349442e-02, 9.41112516e-04, 1.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       ...,
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        1.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 1.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 1.00000000e+00]])

In [59]:
name_to_index = {name: indexx for indexx, name in list(zip(genres.movieId, genres.title))}

In [66]:
def recommend_movies_based_on_plot(movie_input):
    movie_index = name_to_index[movie_input]
    similarity_score = list(enumerate(similarity_matrix[movie_index]))
    similarity_score = sorted(similarity_score, key=lambda x: x[1], reverse=True)
    similarity_score = similarity_score[1:20]
    movie_indices = [i[0] for i in similarity_score]
    return (genres.title.iloc[movie_indices])

In [67]:
recommend_movies_based_on_plot('GoldenEye (1995)')

16342                                  Tron: Legacy (2010)
18435                                     Albatross (2011)
426                                   Calendar Girl (1993)
15347                                      Betrayal (1983)
18211                                       Barocco (1976)
16645    Shogun's Ninja (Ninja bugeicho momochi sandayu...
17037                                  Goddess, The (1958)
381                         Man of No Importance, A (1994)
17325                              Envy (Kiskanmak) (2009)
3434                                         Arthur (1981)
1144                                       Dear God (1996)
136                                 Neon Bible, The (1995)
349                                       Crow, The (1994)
4726                          Max Keeble's Big Move (2001)
4827                              Guadalcanal Diary (1943)
7172                                       Trespass (1992)
10673    Scene at the Sea, A (Ano natsu, ichiban shizuk.

In [77]:
recommend_movies_based_on_plot('Mark of Zorro, The (1940)')

1325                                     Nightwatch (1997)
9074          Puppetmaster, The (Xi meng ren sheng) (1993)
4826                             Halls of Montezuma (1950)
3572                                Puppet Master 4 (1993)
2779                                     Sugar Town (1999)
6991                                       Madhouse (1990)
8983                                    Garden, The (1990)
12875    Colour of Magic, The (Terry Pratchett's The Co...
13182                                          Igor (2008)
2799                                      Guinevere (1999)
11104         Summer in Berlin (Sommer vorm Balkon) (2005)
6091                             Lawless Heart, The (2003)
7357                           Kingdom, The (Riget) (1994)
16125                          Period of Adjustment (1962)
19421               Piranha 3DD (a.k.a. Piranha DD) (2012)
19127                                      Bunnyman (2009)
19136    Dragon Ball: The Curse Of The Blood Rubies (Do.

Видно, что совпадают, как минимум жанры фильмов. Подход сжатия описаний в эмбеддинги даёт какой-то результат.
Использовал материалы https://medium.com/analytics-vidhya/content-based-recommender-systems-in-python-2b330e01eb80